In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
url = "https://www.ccny.cuny.edu/registrar/fall"
response = requests.get(url)


In [14]:
soup = BeautifulSoup(response.text, "html.parser")
soup.prettify()


'<!DOCTYPE html>\n<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="The City College of New York" name="geo.placename"/>\n  <meta content="40.8200471,-73.9492724" name="geo.position"/>\n  <meta content="40.8200471,-73.9492724" name="icbm"/>\n  <meta content="US-NY" name="geo.region"/>\n  <meta content="index, follow" name="robots"/>\n  <link href="https://www.ccny.cuny.edu/registrar/fall" rel="canonical"/>\n  <meta content="Drupal 9" name="generator"/>\n  <meta content="The City College of New York" property="og:site_name"/>\n  <meta content="text" property="og:type"/>\n  <meta content="ht

In [54]:
table = soup.find_all('table')[0]
rows = soup.find_all("tr")
rows

[<tr>
 <th class="text-align-left" scope="col" style="width: 305px;">DATES</th>
 <th class="text-align-left" scope="col" style="width: 327px;">DAYS</th>
 <th scope="col" style="width: 617px;"> </th>
 </tr>,
 <tr>
 <td style="width:305px">
 <p><strong>August 01</strong></p>
 </td>
 <td style="width:327px">
 <p><strong>Sunday</strong></p>
 </td>
 <td style="width:617px">
 <p><strong>Application for degree for January and February 2022 begins</strong></p>
 </td>
 </tr>,
 <tr>
 <td style="width:305px">
 <p>August 18</p>
 </td>
 <td style="width:327px">
 <p>Wednesday</p>
 </td>
 <td style="width:617px">
 <p>Last day to apply for Study Abroad</p>
 </td>
 </tr>,
 <tr>
 <td style="width:305px">
 <p><strong>August 24</strong></p>
 </td>
 <td style="width:327px">
 <p><strong>Tuesday</strong></p>
 </td>
 <td style="width:617px">
 <p><strong>Last day of Registration;<br/>
 			Last day to file ePermit for the Fall 2021;<br/>
 			Last day to drop classes for 100% tuition refund;</strong></p>
 </td>


In [24]:
data = {
    "date": [],
    "day": [],  
    "text": [],
}


In [ ]:
def format_date_range(s: str, default_year: int = 2021):
    s = s.strip()
    # if given like August 1
    if "-" not in s:
        if "," not in s:
            s += f", {default_year}"
        start = end = pd.to_datetime(s)
        return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")

    # if given like Auguest 1-10
    month = s.split()[0]
    day_start, day_end = s.replace(",", "").split()[1].split("-")
    start = pd.to_datetime(f"{month} {day_start}, {default_year}")
    end = pd.to_datetime(f"{month} {day_end}, {default_year}")

    return start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")


# Testing --L> returns a tuple of start and end
print(format_date_range("August 10"))  
print(format_date_range("August 1-10"))  
print(format_date_range("January 01, 2022"))  

('2021-08-10', '2021-08-10')
('2021-08-01', '2021-08-10')
('2022-01-01', '2022-01-01')
2021-08-01 2021-08-10


In [59]:
def get_day(s: str) -> str:
    return s.strip().replace("\n", " ").replace("\t", " ").split()[0]
print(get_day('Wednesday - Friday'))
print(get_day("Friday - Wednesday"))
print(get_day("Saturday"))

Wednesday
Friday
Saturday


In [63]:
# clean text for date information --> remove newlines and tabs and puts it
# into a string with a space as the seperator
def clean_text(s: str) -> str:
    return " ".join(s.strip().split())

print(clean_text('Last day of Registration ' \
'Last day to file ePermit for the Fall 2021 ' \
'Last day to drop classes for 100% tuition refund;'))


Last day of Registration Last day to file ePermit for the Fall 2021 Last day to drop classes for 100% tuition refund;
